In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from os import sys
sys.path.append(r"..")
from utils import DataLoader

### 1. 读取数据

In [2]:
train_path = os.path.join('.', 'data', 'house-prices-advanced-regression-techniques', 'train.csv')
test_path = os.path.join('.', 'data', 'house-prices-advanced-regression-techniques', 'test.csv')
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
# train_data
# print(train_data.shape)
# print(test_data.shape)

### 2. 预处理
对数据做预处理，主要包含如下几个步骤：

1. 对特征进行标准化：0均值，单位方差
2. 缺失值处理：填充为相应特征的平均值
3. 处理离散值：可以使用独热编码

In [3]:
all_features = pd.concat([train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]])
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)
#type(numeric_features)
#print(all_features.shape)

In [4]:
all_features = pd.get_dummies(all_features, dummy_na=True)

### 3. 将Pandas数据转为NumPy格式，并转换为张量表示

In [5]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].to_numpy(), dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].to_numpy(), dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.to_numpy().reshape(-1, 1), dtype=torch.float32)

### 4.训练

In [11]:
loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

""" log RMSE, 对于较大值有压缩效果 """
def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = DataLoader.load_array((train_features, train_labels), batch_size)
    """ using Adam """
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay= weight_decay)
    for epoch in range(num_epochs):
        for X , y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

### 5.K折交叉验证

In [12]:
def get_k_folder_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)  # get index of valid set
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [13]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_folder_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        print(f'folder {i + 1}: train log rmse {float(train_ls[-1]):f}, '
              f'valid log rmse {float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [27]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
print(f'{k}-folder validation: average train log rmse: {float(train_l):f}, '
      f'average test log rmse: {float(valid_l):f}')

folder 1: train log rmse 0.170098, valid log rmse 0.157322
folder 2: train log rmse 0.162141, valid log rmse 0.192991
folder 3: train log rmse 0.163526, valid log rmse 0.168239
folder 4: train log rmse 0.168539, valid log rmse 0.154960
folder 5: train log rmse 0.163549, valid log rmse 0.183514
5-folder validation: average train log rmse: 0.165571, average test log rmse: 0.171405


### 6. 正式预测

In [29]:
def train_and_pred(net, train_features, train_labels, test_data, num_epochs,
                   learning_rate, weight_decay, batch_size):
    train_ls, _ = train(net, train_features, train_labels, None, None, num_epochs, learning_rate,
                        weight_decay, batch_size)
    print(f'Train log rmse: {float(train_ls[-1]):f}')
    preds = net(test_features).detach().numpy()
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv('submission.csv', index=False)

train_and_pred(get_net(), train_features, train_labels, test_data, num_epochs,
               lr, weight_decay, batch_size)
    

Train log rmse: 0.162599
